In [12]:
import numpy as np
import math
import scipy.linalg as la

# Semana 2 PIBIC

## Gerando Tabelas Verdade (Truth Tables)

Para a geração de tabelas verdade com n inputs no circuito, com $n \ge 1$. Esta é uma função recursiva que gera uma tabela verdade em forma de matriz para n inputs arbitrários.

Talvez existam formas mais rápidas e menores de fazê-las, mas esta função é mais legível para os usuários e para os parceiros da pesquisa. Modifiquei o nome das variáveis para auxiliar no processo.

Além disso, modifiquei-a para que retornasse um numpy array ao invés de um array convencional do python, aumentando a eficiência da função visto que numpy arrays são mais rápidos de se fazer modificações na função e até mesmo posteriores.

In [13]:
"""
DOCUMENTAÇÃO DA FUNÇÃO:

Retorna uma tabela verdade (truth_table) dado um circuito com n inputs maiores ou iguais a 1
Checa se a variável number_of_inputs é inteira. Se sim, continua normalmente, se não lança uma exceção com a mensagem
'Parameter \'number_of_inputs\' must be a integer'

:number_of_inputs: número de entradas do circuito. Ditarão a quantidade de colunas da tabela verdade
"""

def TruthTable(NumberOfInputs):
    if not isinstance(NumberOfInputs, int):
        raise ValueError('Parameter \'number_of_inputs\' must be a integer') 
    if NumberOfInputs < 1:
        return [[]]
    
    SubTable = TruthTable(NumberOfInputs - 1)
    return [row + [v] for row in SubTable for v in [0,1]]

In [14]:
table = np.array(TruthTable(3))
table

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0],
       [1, 1, 1]])

## Modulação PAM

Quando tratamos de tabelas verdade de circuitos eletrônicos, muitas vezes precisamos converter esses sinais em Modulação por Amplitude do Pulso (PAM: Pulse Amplitude Moduling) para realizarmos cálculos ou verificações e depois retornamos a representação do circuito a meneira original, com 0's e 1's.

Para isso, convertemos os campos (labels) das tabelas verdades por $1+\epsilon$ caso o valor no campo seja 1 ou $1-\epsilon$ caso seja 0. A matriz resultante é chamada de $\textbf{Imp}$.

No exemplo a seguir, utilizaremos a variável table definida no código acima para os testes e o valor $\epsilon = 5$:

In [23]:
"""
DOCUMENTAÇÃO DA FUNÇÃO:

Função que retorna uma modulação PAM para uma dada tabela verdade e certo epsilon real arbitrário
Não modifica a função original, realiza alterações em uma cópia e a retorna no final
Functiona para qualquer tabela verdade (truth_table) com n inputs

:truth_table: tabela verdade a ser analisada
:epsilon: epsilon no qual os valore serão calculados
"""

def PAM(Truthtable, epsilon):
    PamTable = np.array(Truthtable.copy())
    rows, columns = PamTable.shape
    for row in range(rows):
        for column in range(columns):
            if PamTable[row][column] == 1:
                PamTable[row][column] = 1 + epsilon
            else:
                PamTable[row][column] = 1 - epsilon
    return PamTable

In [24]:
Imp = PAM(table, 5)
print(f'IMP: \n{Imp}\n\nTABELA VERDADE ORIGINAL: \n{table}')

IMP: 
[[-4 -4 -4]
 [-4 -4  6]
 [-4  6 -4]
 [-4  6  6]
 [ 6 -4 -4]
 [ 6 -4  6]
 [ 6  6 -4]
 [ 6  6  6]]

TABELA VERDADE ORIGINAL: 
[[0 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 1]
 [1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]


Note que a linha de código dada por: 

``PAM_table = truth_table.copy()``

faz com que a variável PAM_table seja uma cópia da truth_table, não alterando assim a tabela verdade inicial recebdia como parãmetro. Isso é importante pois podemos realizar os cálculos necessários com a variável Imp que guarda o valor de retorno da função sem perder a tabela verdade original (Desfaz a necessidade da função $\textbf{InvPAM}$).

## Geração de Tabelas Verdade Padrões (OR, AND)

Para dar continuidade à pesquisa, precisamos de uma função que nos retorne tabelas verdade de operadores lógicos padrões (OR, AND etc...).

In [25]:
"""
DOCUMENTAÇÃO DA FUNÇÃO:

Retorna uma tabela verdade do circuito lógico OR ou AND de acordo com uma tabela verdade (truth_table) e quantidade
de inputs (number_of_inputs). Checa se a tabela está vazia, se sim a retorna (vazia), se não realiza os cálculos.

params: truth_table, number_of_inputs, boolean_expression (OR, AND)
"""

def OrAndTruthTableGenerator(TruthTable, BooleanOperator):
    rows, columns = TruthTable.shape
    BooleanOperator = BooleanOperator.lower()
    ResultTable = np.zeros(shape=(rows, 1), dtype=int)
    
    if TruthTable.size == 0:
        return TruthTable
    
    if BooleanOperator == 'or':
        for row in range(rows):
            BooleanRowArray = TruthTable[row][:]
            if np.any(BooleanRowArray):
                ResultTable[row][0] = 1
    elif BooleanOperator == 'and':
        for row in range(rows):
            BooleanRowArray = TruthTable[row][:]
            if np.all(BooleanRowArray):
                ResultTable[row][0] = 1;
    else:
        raise ValueError('Parameter boolean_expression must be \'or\' or \'and\'')
        
    return ResultTable

In [26]:
OR = OrAndTruthTableGenerator(table, 'or')
AND = OrAndTruthTableGenerator(table, 'and')
print(f'OR TRUTH TABLE: \n{OR} \n\nAND TRUTH TABLE: \n{AND}')

OR TRUTH TABLE: 
[[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]] 

AND TRUTH TABLE: 
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]]


Perceba que os exemplos utilizados para os cálculos das funções são feitos utilizando tabelas verdade de 3 inputs. No entanto, como já documentado, essas funções aceitam tabelas verdade de n inputs, sendo n um número natural.

## Código de Teste

Os códigos abaixo realizam testes com as funções abordadas, seguindo o modelo (exemplo para o arranjo triangular):

$A = ArranjoT rinagular(α, β, γ) \newline T = MatrizTransferencia(A) \newline Inp = PAM(TV(3)) \newline Out = T Inp^{T} \newline Out = Out^{T} \newline OutPAM = PAM(OR)$

In [19]:
def ArranjoTriangular(alpha, beta, gama):
    A = np.array([[0, complex(0, gama), complex(0, beta)], 
                  [complex(0, gama), 0, complex(0, alpha)], 
                  [complex(0, beta), complex(0, alpha), 0]])
    return A

In [27]:
def MatrizTransferenciaTriangular(A):
    eigvals, eigvecs = la.eig(A)
    D = np.array([[eigvals[0], 0, 0], 
                  [0, eigvals[1], 0], 
                  [0, 0, eigvals[2]]])
    P = eigvecs.copy()
    E = math.e**D
    
    # T é a matriz de transferência
    T = P @ E @ la.inv(P)
    
    return T

In [33]:
A = ArranjoTriangular(1, 2, 3)
T = MatrizTransferenciaTriangular(A)
Inp = PAM(TruthTable(3), 5)
Out = T @ np.transpose(Inp)
Out = np.transpose(Out)
OutPAM = PAM(OR, 5)

In [36]:
print(f'A: \n{A}\n\nT: \n{T}\n\nInp: \n{Inp}\n\nOut: \n{Out}\n\nOutPAM: \n{OutPAM}')

A: 
[[0.+0.j 0.+3.j 0.+2.j]
 [0.+3.j 0.+0.j 0.+1.j]
 [0.+2.j 0.+1.j 0.+0.j]]

T: 
[[-0.14870738-0.33599426j -0.4225552 -0.3998916j   1.29009591-0.18361063j]
 [-0.4225552 -0.3998916j  -1.10731104-0.48536506j -1.15369545+0.13225087j]
 [ 1.29009591-0.18361063j -1.15369545+0.13225087j  0.30658844-0.73431637j]]

Inp: 
[[-4 -4 -4]
 [-4 -4  6]
 [-4  6 -4]
 [-4  6  6]
 [ 6 -4 -4]
 [ 6 -4  6]
 [ 6  6 -4]
 [ 6  6  6]]

Out: 
[[ -2.87533333+3.67798595j  10.73424675+3.01202312j
   -1.77195563+3.1427045j ]
 [ 10.02562579+1.84187969j  -0.80270773+4.33453186j
    1.29392882-4.20045924j]
 [ -7.10088531-0.32093003j  -0.33886366-1.84162744j
  -13.30891011+4.46521324j]
 [  5.80007381-2.15703629j -11.87581815-0.5191187j
  -10.24302567-2.8779505j ]
 [ -4.36240714+0.31804331j   6.50869477-0.98689286j
   11.12900348+1.30659825j]
 [  8.53855197-1.51806295j  -5.02825971+0.33561588j
   14.19488793-6.03656549j]
 [ -8.58795912-3.68087267j  -4.56441564-5.84054342j
   -0.407951  +2.62910698j]
 [  4.31299999-5.51697